# Geomorphons

This notebook computes geomorphons rasters of various scales to the entire Trinity Basin.

The Trinity Basin is a huc8 watershed.  It is too large to process in one chunk.  Here we will break it up into huc10 basins and process those, then join them as a vrt.

In keeping consistent with the methods of Kane et al.\cite{kane2015} we will use $r_{outer}$ of 100 m, 250 m, 500 m, 1000 m, and 2000 m outer radii. Since the authors do not specify the inner radius used, here we will use the rule that $r_{inner} = \frac{r_{outer}}{2}$.  A flatness threshod of 0 is used.


In [16]:
import whitebox
wbt = whitebox.WhiteboxTools()

from pathlib import Path
import os
import temppathlib

import geopandas as gpd
from tqdm import tqdm
import xarray as xr
import rioxarray
from rioxarray.exceptions  import NoDataInBounds

In [9]:
# path to input files
dem_path = Path('/home/michael/tmp/trinity_basin_carrhirzdelta_dem.tif')
huc10_path = Path('/media/storage/watershed_boundaries/trinity_river_huc10.geojson')

#path for output
out_dir = Path.cwd().parent / 'geomorphons'
os.makedirs(out_dir, exist_ok=True)

In [3]:
def my_callback(value):
    #if not '%' in value:
    #print(value)
    pass


def geomorph_all_scales(dem_path, out_dir, basin):
    for r in tqdm([100, 250, 500, 1000, 2000]):

        output = out_dir / f'geomorph_{basin}_{r}.tif' 
        
        wbt.geomorphons(
            dem_path, 
            output, 
            search=r, 
            threshold=0.0, 
            fdist=0, 
            skip=round(r / 2), 
            forms=True, 
            residuals=False, 
            callback=my_callback
        )

In [4]:
# read huc10s to df
huc10s = gpd.read_file(huc10_path)[['areasqkm', 'name', 'huc10', 'geometry']]
huc10s.head()

,areasqkm,name,huc10,geometry
0,302.85,Coffee Creek,1801021101,"MULTIPOLYGON (((514743.340 4562033.250, 514780..."
1,410.33,Tangle Blue Creek-Trinity River,1801021102,"MULTIPOLYGON (((536306.121 4579423.347, 536324..."
2,300.82,East Fork Trinity River,1801021103,"MULTIPOLYGON (((537372.506 4564164.410, 537418..."
3,357.19,Stuart Fork,1801021104,"MULTIPOLYGON (((501808.643 4540250.013, 501812..."
4,489.90,Swift Creek-Trinity River,1801021105,"MULTIPOLYGON (((527863.541 4548319.278, 527902..."


In [13]:
row.huc10

'1801021112'

In [14]:
basin_files = list(out_dir.glob('*1801021101*'))
len(basin_files)

5

In [5]:
with temppathlib.TemporaryDirectory() as tmp_dir:
    for i, row in huc10s.iterrows():
        print(f'on row {i} of {len(huc10s)}')   
        # get bounds of basin
        minx, miny, maxx, maxy = row.geometry.bounds
        # read DEM clipped to bounds
        try:
            dem = rioxarray.open_rasterio(dem_path).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy
            )
        except NoDataInBounds:
            print(f'{row.huc10} ({row.name}) failed with NoDataInBounds')
            continue
            
        # clip to 100m buffered huc10 basin
        dem = dem.rio.clip([row.geometry.buffer(100)])
        # write a tmp dem
        dst = tmp_dir.path / f'{row.huc10}.tif'
        dem.rio.to_raster(dst)
        # delete dem to free memory
        del dem
        # make geomorphons of all scales
        geomorph_all_scales(dst, out_dir, row.huc10)
    

./whitebox_tools --run="Geomorphons" --dem='/tmp/tmp4wihpzf4/1801021101.tif' --output='/home/michael/TreeMortality/geomorphons/geomorph_1801021101_100.tif' --search=100 --threshold=0.0 --fdist=0 --skip=50 --forms -v --compress_rasters=False

****************************
* Welcome to Geomorphons   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Generating global ternary codes...
Computing geomorphons...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
P

NoDataInBounds: No data found in bounds.

In [8]:
it =  huc10s.iterrows()

In [10]:
_, row = next(it)
minx, miny, maxx, maxy = row.geometry.bounds


(516571.2758170952, 4546883.022268877, 543585.0324151008, 4579762.581275195)